In [23]:
from selenium import webdriver

import time

import pandas as pd

from bs4 import BeautifulSoup

import random

url = r' ' #给出链接

driver = webdriver.Firefox()

driver.get(url)

blank_ISSN = driver.find_element_by_id('magazine_value1')

#id是在源码中搜索而来

blank_ISSN.send_keys('0577-9154') #输入内容，期刊的ISSN号码

buttom_search = driver.find_element_by_id('btnSearch') #找到搜索按钮

buttom_search.click() #点击搜索

time.sleep(2) #停一小会儿

driver.switch_to.default_content() #找到iframe子页面

driver.switch_to.frame('iframeResult') #切换到iframe子页面

#一次请求50篇论文信息

button_50=driver.find_element_by_xpath(r'/html/body/form/table/tbody/tr[1]/td/table/tbody/tr[1]/td/table/tbody/tr/td[2]/div[2]/a[3]')

button_50.click()

htmls = [] #遍历所有子页面，储存到此列表

b = 1

while True:

    try:

        soup = BeautifulSoup(driver.page_source,'html.parser')

        htmls.append(soup)

   #储存完成，点击下一页

        time.sleep(random.randint(3,6)) #暂停3~6s，这样封IP概率更小

        button_next = driver.find_element_by_id('Page_next')

        button_next.click()

    except :

        print('Error{}'.format(b))

        time.sleep(60)

        b = b+1
        transfer_list=[] #存储html源码的中转列表

for soups in htmls:

    for i in soups.find_all('tr')[7:57]: #在soup搜索出来的tr标签中，列表头尾都有冗余信息，删掉不要

        transfer_list.append(i)

        raw_paper_infos = [] #使用正则匹配的思路：第一步，缩小信息所在的范围；第二步，精确搜索并删除多余的字符。下面进行第一步

for i in transfer_list: #遍历transfer_list中的每一个tr标签，其结构：[[一个tr标签包含的源码],[].……]

    paper = str(i)

    title = re.findall('target="_blank">.*</a>',paper)

    authors = re.findall('target="knet">.*</a>',paper)

    pub_year = re.findall('....-..-..',paper)

    download_url = re.findall('briefDl_D" href=".*pdfdown',paper)

    quoted_num1 = re.findall('scrollbars=yes.*</span',paper)

    download_num = re.findall('onclick="SubCountDownLoad.*</a>',paper)

    raw_paper_infos.append([title,authors,pub_year,download_url,quoted_num1,download_num]) #储存了初步的匹配结果

#接下来继续匹配工作，精确匹配和删除多余字符：

    papers = [] #用于存放最终干净的匹配结果

    illegal_word = ['/','\',''',':','|','?','*'] #考虑到后续下载时，保存文件时会有一些非法字符导致文件名生成失败（windows不允许这些字符出现在文件名中）

for i in raw_paper_infos:

    if i!=[[], [], [], [], [], []]: #初步搜索时候，会有一些tr标签不是我们的论文标签，不符合初步搜索的要求，其结果就是空列表，通过判断过滤掉

        title2 = i[0][0].strip('target="_blank>~</a>')

for x in illegal_word:

    title2.replace(x,"")

    authors2 = '，'.join(j for j in re.findall('[\u2E80-\u9FFF]+',str(i[1])))

    pub_year2 = i[2][0]

    download_url2 = (i[3][0].strip('briefDl_D" href="..')).replace(';','&') #按照';'连接为caj格式，'&'连接为pdf格式

if i[4] == []:

    quoted_num2 = int(0)

else:

    quoted_num2 = int(re.findall(r'>.*</a',str(i[4][0]))[0].strip('></a'))

#被引次数对于发表不久的论文，可能没有这一项的数据，因此对于匹配不到的赋值0



download_num2 = int(re.findall(r'>.*<',i[5][0])[0].strip('><')) #论文得下载次数

papers.append([title2,authors2,pub_year2,download_url2,quoted_num2,download_num2])

#到此为止，我们有了全部所需要的论文信息。

#储存到excel

pd.DataFrame(papers).to_excel(r'.\JJYJ论文信息汇总.xlsx')

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 
